# Interactions between patchy macro-ions

In this exercise we will use the Metropolis-Hastings Monte Carlo algorithm to study how charged, patchy particles interact in an aquous salt solution. The model is as follows:

- The spherical macro-ions have a radius of 40 Å and have a fixed number of positive and negative charges on it's surface. These can be uniformly distributed (`P00.xyz`) which gives a net-charge or _monopole moment_ of -8e. They can also be unevenly distributed (`P18.xyz`) to give, in addition to the monopole moment, also a _dipole moment_.

- The solvent, water, is treated as a dielectric continuum

- Counter-ions and background electrolyte is also _implicit_ and described with a so-called Yukawa potential which can be derived from Debye-Hückel theory.

To learn more about the input and output, see the [online manual](https://faunus.readthedocs.io).

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import nglview as nv
import mdtraj as md
import json

## Task: Get familiar with the input and output

The simulation is controlled by the input file `input.yml` and the shell command below sends it to the Faunus program. Get familiar with the input (`water.yml`) and output (`out.json`):

1. What Monte Carlo moves do we have?
1. What does _acceptance_ in the output mean?
1. At the end of the simulation, you get a _drift_. What does that tell you?
1. Is it OK to have a non-zero system charge?
1. How does the _acceptance_ vary with the so-called _displacement parameter_?
1. Plot `energy.dat` and describe how the system energy develops with MC steps?
1. Plot the radial distribution function, $g(r)$ and give an interpretation.
1. Give a mathematical formula for the used pair-potential
1. Things to play with:
  - [ ] number of MC steps
  - [ ] salt concentration
  - [ ] investigate P00-P00, P00-P18, and P18-P18 combinations.
  - [ ] add multipole analysis -- see manual
  - [ ] add a scattering intensity analysis -- see manual. You may also want to add more molecule.
    
### Running and restarting simulations

When starting a simulation, the molecules are place at random positions and orientations. At the end of the simulation, a file `state.json` will be saved, and you can use it as the starting point for a new simulation by adding `-s state.json` after `faunus` shell command. You can also ask for more information using the `--verbosity` command line argument.


In [ ]:
!/Users/mikael/github/faunus/scripts/yason.py input.yml | /Users/mikael/github/faunus/cmake-build-release-gcc11/faunus # run simulation!

## Task: Visualize simulation trajectory

During simulation, the configurations are periodically saved to disk and you can control this in the `analysis` section. Here we use the module [`nglview`](https://github.com/nglviewer/nglview) to visualise this directly in the notebook. Consult the nglview documentation and try to customise the molecular representation. Describe what happens to the water molecules during the simulation.

- [ngl selection language](https://nglviewer.org/ngl/api/manual/selection-language.html)
- [ngl tutorial](https://osscar-docs.readthedocs.io/en/latest/visualizer/index.html)
- [Advanced ngl](https://projects.volkamerlab.org/teachopencadd/talktorials/T017_advanced_nglview_usage.html)

In [ ]:
traj = md.load('traj.xtc', top='confout.gro')
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_spacefill(selection='*', radius=2.0)
view.add_spacefill(selection='.MP', radius=100, color='white') # larger?
view.background = 'white'
view.camera = 'orthographic'
view.center()
view.add_unitcell()
view

## Tips and Tricks

Information and statistics about the simulation is stored in `out.json`. Python can easily read JSON files as shown below. The output file can be fairly nested, so it's always a good idea to first inspect it by double-clicking it in the file browser.

In [ ]:
with open('out.json') as file:
    d = json.load(file) # --> dict
    acceptance = d['moves'][0]['moltransrot']['acceptance']
    print('Acceptance probability for the first move is {:.1f}%'.format(acceptance*100))

#### Plotting

We can also plot data using `matplotlib`. By first inspecting the `energy.dat` file in the file browser, we see that it should be read like this:

In [ ]:
step, total, selfenergy, nonbonded = np.loadtxt('energy.dat', unpack=True, skiprows=1)
print('average total energy = {:.3E} kT'.format(total.mean()))

In [ ]:
plt.plot(step, total, label='total')
plt.plot(step, selfenergy, label='self energy')
plt.plot(step, nonbonded, label='nonbonded')
plt.xlabel('Monte Carlo steps')
plt.ylabel(r'Energy ($k_BT)$')
plt.title('Energy vs. steps')
plt.legend(loc=0, frameon=False)